<a href="https://colab.research.google.com/github/Raniamea/arabic-video-summarisation/blob/main/notebooks/03_ArabicPreprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Arabic Preprocessing with CAMeL Tools
This notebook performs Arabic text preprocessing using CAMeL Tools, including normalization & lemmatization

In [ ]:

# Install compatible versions of NumPy and CAMeL Tools
!pip install numpy==1.23.5 --force-reinstall --no-cache-dir
!pip install camel-tools

In [ ]:
# Download the morphology DB for MSA
!camel_data -i morphology-db-msa-r13

# Download the MLE disambiguator for MSA
!camel_data -i disambig-mle-calima-msa-r13

!mkdir -p ~/.camel_tools/data/ner/arabert
!wget -q https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-msa-ner/resolve/main/config.json -O ~/.camel_tools/data/ner/arabert/config.json
!wget -q https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-msa-ner/resolve/main/pytorch_model.bin -O ~/.camel_tools/data/ner/arabert/pytorch_model.bin
!wget -q https://huggingface.co/CAMeL-Lab/bert-base-arabic-camelbert-msa-ner/resolve/main/vocab.txt -O ~/.camel_tools/data/ner/arabert/vocab.txt


In [ ]:
from google.colab import drive
import os

# Unmount first
!fusermount -u /content/drive || echo "Already unmounted"

# Delete the mount folder entirely
!rm -rf /content/drive

# Now mount again
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os, json, re
from typing import List, Dict, Any

# Project paths
base_path = "/content/drive/MyDrive/ArabicVideoSummariser"
param_path = os.path.join(base_path, "params.json")
with open(param_path, "r", encoding="utf-8") as f:
    params = json.load(f)
video_filename = params.get("video_file")
video_filename="Almasbagha.mp4"

assert video_filename, "params.json must include 'video_file'."
video_name = os.path.splitext(video_filename)[0]
videos_path = os.path.join(base_path, "videos")
transcripts_path = os.path.join(base_path, "transcripts")
preprocessed_path= os.path.join(base_path, "Preprocessed")
video_path = os.path.join(videos_path, video_filename)
assert os.path.exists(video_path), f"Video file not found: {video_path}"
transcript_path = os.path.join(transcripts_path, f"{video_name}_ar.json")
preprocessed_transcript_path= os.path.join(preprocessed_path, f"{video_name}_CleanTranscript.json")

print("Input :", transcript_path)
print("Output:", preprocessed_transcript_path)


In [ ]:
# Remove diacritics and standardize common forms (keep things readable but consistent)
_AR_DIAC = re.compile(r"[\u0610-\u061A\u064B-\u065F\u0670\u06D6-\u06ED]")
_TATWEEL = "\u0640"

def normalize_ar(text: str) -> str:
    if not text:
        return ""
    s = text
    s = _AR_DIAC.sub("", s)         # strip diacritics
    s = s.replace(_TATWEEL, "")     # strip tatweel
    # Normalize hamza/alef variants
    s = s.replace("أ", "ا").replace("إ", "ا").replace("آ", "ا")
    # Normalize alef maqsura to ya
    s = s.replace("ى", "ي")
    # Collapse whitespace
    s = re.sub(r"\s+", " ", s).strip()
    return s

def simple_word_tokens(text: str) -> List[str]:
    # Arabic letters + Arabic/ASCII digits
    return re.findall(r"[\u0600-\u06FF]+|[0-9٠-٩]+", text)

In [ ]:
from camel_tools.disambig.mle import MLEDisambiguator

# Pretrained MSA disambiguator
msa_mle = MLEDisambiguator.pretrained()

from camel_tools.ner import NERecognizer
ner = NERecognizer(model_path="/root/.camel_tools/data/ner/arabert/")

def dediac(s: str) -> str:
    return _AR_DIAC.sub("", s or "")

def _analysis_dict_from_scored(best):
    # CAMeL returns ScoredAnalysis objects; the dict is in .analysis
    if hasattr(best, "analysis"):
        return best.analysis
    if isinstance(best, dict):
        return best
    if hasattr(best, "__dict__"):
        return best.__dict__
    return {}

def disambiguate_tokens(tokens: List[str]) -> Dict[str, List[str]]:
    """Return parallel lists: lemmas, pos, diac, plus small diagnostics."""
    lemmas, poss, diacs = [], [], []
    oov_flags = []  # True if we failed to get a proper analysis
    results = msa_mle.disambiguate(tokens)
    for tok, res in zip(tokens, results):
        lemma, pos, diac = tok, "", ""
        oov = True
        try:
            if getattr(res, "analyses", None):
                best = res.analyses[0]
                feats = _analysis_dict_from_scored(best)
                raw_lemma = feats.get("lemma") or feats.get("lex") or tok
                lemma = dediac(raw_lemma)
                pos   = feats.get("pos") or feats.get("bw") or ""
                diac  = feats.get("diac") or ""
                oov   = False if (pos or diac or raw_lemma) else True
        except Exception:
            pass
        lemmas.append(lemma); poss.append(pos); diacs.append(diac); oov_flags.append(oov)
    return {"lemmas": lemmas, "pos": poss, "diac": diacs, "oov": oov_flags}


In [ ]:
from collections import defaultdict

class DiacSmoother:
    """
    Keeps a canonical diacritization per (lemma, POS) across the document
    to reduce flip-flops. Conservative: only applies when POS matches and we
    already have a non-empty canonical diac for that (lemma, POS).
    """
    def __init__(self):
        self.canonical = {}                    # (lemma, pos) -> diac
        self.freqs = defaultdict(lambda: defaultdict(int))  # (lemma,pos)[diac] -> count

    def observe(self, lemma: str, pos: str, diac: str):
        if not lemma or not pos or not diac:
            return
        key = (lemma, pos)
        self.freqs[key][diac] += 1
        # Update canonical to the most frequent diac so far
        self.canonical[key] = max(self.freqs[key], key=self.freqs[key].get)

    def smooth(self, lemma: str, pos: str, diac: str) -> str:
        if not lemma or not pos:
            return diac
        key = (lemma, pos)
        canon = self.canonical.get(key)
        if canon and diac and diac != canon:
            # Prefer the canonical diac unless current diac is empty
            return canon
        if canon and not diac:
            # Fill in missing diac with canonical
            return canon
        return diac

smoother = DiacSmoother()


In [ ]:
# Load transcript; accept either {"segments":[...]} or a plain list
with open(transcript_path, "r", encoding="utf-8") as f:
    raw = json.load(f)

segments = raw.get("segments", raw)
assert isinstance(segments, list), "Input JSON must be a list or have a 'segments' list."

clean_segments = []

import json, os
from camel_tools.ner import NERecognizer

# --- Load transcript (accepts dict or list) ---
with open(transcript_path, "r", encoding="utf-8") as f:
    raw = json.load(f)

segments = raw.get("segments", raw)
assert isinstance(segments, list), "Input JSON must be a list or have a 'segments' list."

# --- Load NER model once ---
ner = NERecognizer.pretrained()

clean_segments = []

def _normalize_to_list(val):
    if isinstance(val, list):
        return val
    if isinstance(val, str):
        val = val.strip()
        return [val] if val else []
    return []

def _is_only_musiqi(seg):
    """Return True if the segment has only one lemma equal to 'موسيقي'."""
    lem = _normalize_to_list(seg.get("lemmas"))
    return len(lem) == 1 and lem[0].strip() == "موسيقي"

# --- Preprocess each segment ---
for seg in segments:
    start = seg.get("start") if isinstance(seg, dict) else None
    end   = seg.get("end")   if isinstance(seg, dict) else None
    text  = (seg.get("text") or seg.get("original") or seg.get("utterance") or "").strip()

    # --- Normalization ---
    text_norm = normalize_ar(text)

    # --- Tokenization ---
    tokens = simple_word_tokens(text_norm)

    if tokens:
        ana = disambiguate_tokens(tokens)
    else:
        ana = {"lemmas": [], "pos": [], "diac": [], "oov": []}

    # --- Diacritic smoothing ---
    diac_smooth = []
    for l, p, d in zip(ana["lemmas"], ana["pos"], ana["diac"]):
        if d:
            smoother.observe(l, p, d)
        diac_smooth.append(smoother.smooth(l, p, d))

    # --- Segment quality heuristics ---
    n = max(1, len(tokens))
    diac_cov = sum(1 for d in diac_smooth if d) / n
    oov_ratio = sum(1 for o in ana["oov"] if o) / n

    # --- Named Entity Recognition ---
    ner_tags = ner.predict_sentence(tokens) if tokens else []

    # --- Extract named entities ---
    def extract_named_entities(tokens, tags):
        entities = []
        current_entity, current_type = [], None
        for token, tag in zip(tokens, tags):
            if tag != 'O':
                if tag == current_type:
                    current_entity.append(token)
                else:
                    if current_entity:
                        entities.append((' '.join(current_entity), current_type))
                    current_entity = [token]
                    current_type = tag
            else:
                if current_entity:
                    entities.append((' '.join(current_entity), current_type))
                    current_entity, current_type = [], None
        if current_entity:
            entities.append((' '.join(current_entity), current_type))
        return entities

    named_entities = extract_named_entities(tokens, ner_tags)

    clean_segments.append({
        "start": start,
        "end": end,
        "original": text,
        "text_norm": text_norm,
        "tokens": tokens,
        "lemmas": ana["lemmas"],
        "pos": ana["pos"],
        "diac": diac_smooth,
        "diac_coverage": round(diac_cov, 3),
        "oov_ratio": round(oov_ratio, 3),
        "ner": ner_tags,
        "named_entities": named_entities,
    })

# --- Merge consecutive 'موسيقي' segments ---
merged_segments = []
i = 0
merged_groups = 0

while i < len(clean_segments):
    seg = clean_segments[i]

    if not _is_only_musiqi(seg):
        merged_segments.append(seg)
        i += 1
        continue

    # Start group of consecutive 'موسيقي' segments
    start_time = seg.get("start")
    end_time = seg.get("end")
    j = i + 1

    while j < len(clean_segments) and _is_only_musiqi(clean_segments[j]):
        end_time = clean_segments[j].get("end")
        j += 1

    # Replace group with one empty placeholder
    merged_groups += 1
    merged_segments.append({
        "start": start_time,
        "end": end_time,
        "text": "",
        "text_norm": "",
        "tokens": [],
        "lemmas": [],
        "pos": [],
        "diac": [],
        "diac_coverage": 0.0,
        "oov_ratio": 0.0,
        "ner": [],
        "named_entities": [],
        "placeholder": True
    })

    i = j

# --- Final structured output ---
clean_doc = {
    "video": video_name,
    "num_segments": len(merged_segments),
    "segments": merged_segments
}


with open(preprocessed_transcript_path, "w", encoding="utf-8") as f:
    json.dump(clean_doc, f, ensure_ascii=False, indent=2)

print(f"Processed {len(segments)} → {len(merged_segments)} segments")
print(f"Merged groups of consecutive 'موسيقي': {merged_groups}")
print(f" Saved new file: {preprocessed_transcript_path}")

